In [1]:
import os
from abc import ABC, abstractmethod

## Document Cleaner With (.txt, .json, .html, .csv) Supporting Extension
#### below you can create more concreate class if you wana to add more other Extention like docx and all
### Note - Use Document Cleaner class only if you want only content into your RAG PipeLine, cause if you use HTMLCleaner then it will clean all css scrpit it will only leave the content which are shown in screen also remove the empty tag i strongly recommoned you to please go through the README before using this files.

In [2]:
# Abstract class for cleaning documents
class AbstractDocumentCleaner(ABC):
    @abstractmethod
    def clean_document(self, my_documents: list) -> list:
        pass

In [3]:
class CSVCleaner(AbstractDocumentCleaner):
    def clean_document(self, my_file:str)->str:
        import pandas as pd    

        df = pd.read_csv(my_file)

        df.columns = df.columns.str.strip()
        for col in df.columns:
            if pd.api.types.is_numeric_dtype(df[col]):
                df[col].fillna(0, inplace=True)
            else:
                df[col].fillna('', inplace=True)
                df[col] = df[col].str.strip()

        df = df.drop_duplicates()

        df.to_csv(my_file, index=False)
        
        return f"CSV file '{my_file}' has been cleaned in place."

In [4]:
class HTMLCleaner(AbstractDocumentCleaner):
    def clean_document(self, my_file: str) -> str:
        from bs4 import BeautifulSoup, Comment
        
        with open(my_file, 'r', encoding='utf-8') as file:
            content = file.read()

        soup = BeautifulSoup(content, 'html.parser')

        # Remove comments
        for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
            comment.extract()

        # Remove script and style tags
        for tag_name in ['script', 'style']:
            for tag in soup.find_all(tag_name):
                tag.decompose()

        # Remove leading and trailing whitespace from tag contents
        for tag in soup.find_all(True):  # True finds all tags
            if tag.string:
                tag.string = tag.string.strip()

        # Remove blank tags or tags with only whitespace
        for tag in soup.find_all(True):
            if not tag.contents or all(str(content).isspace() for content in tag.contents):
                tag.decompose()

        # Writing clean data back to the file
        with open(my_file, 'w', encoding='utf-8') as file:
            file.write(str(soup.prettify()))

        return f"HTML file '{my_file}' has been cleaned in place."

In [5]:
class JSONCleaner(AbstractDocumentCleaner):
    def clean_document(self, my_file: str) -> str:
        import json
        with open(my_file, 'r', encoding='utf-8') as file:
            data = json.load(file)

        # Clean the JSON data
        def clean_json(d):
            if isinstance(d, dict):
                return {k.strip(): clean_json(v) for k, v in d.items()}
            elif isinstance(d, list):
                return [clean_json(i) for i in d]
            elif isinstance(d, str):
                return d.strip()
            else:
                return d

        cleaned_data = clean_json(data)

        # Writing clean data back to the file
        with open(my_file, 'w', encoding='utf-8') as file:
            json.dump(cleaned_data, file, indent=4, separators=(',', ': '))

        return f"JSON file '{my_file}' has been cleaned in place."

In [6]:
class TXTCleaner(AbstractDocumentCleaner):
    def clean_document(self, my_file: str) -> str:
        with open(my_file, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        cleaned_lines = [line.strip() for line in lines if line.strip()]

        with open(my_file, 'w', encoding='utf-8') as file:
            file.write('\n'.join(cleaned_lines))

        return f"Text file '{my_file}' has been cleaned in place."

In [7]:
class DocumentCleaner:
    
    def __init__(self):
        
        self.cleaner_mapping = {
            '.csv': CSVCleaner(),
            '.html': HTMLCleaner(),
            '.json': JSONCleaner(),
            '.txt': TXTCleaner(),
        }

    def clean(self, my_file_path_or_api: list):
        try:
            file_extension = os.path.splitext(my_file_path_or_api)[1].lower()
            loader = self.cleaner_mapping[file_extension]
            if loader is None:
                raise ValueError(f"Unsupported File type: {file_extension}")

            print(loader.clean_document(my_file_path_or_api))

        except ValueError as ve:
            print(f"ValueError : {ve}")
            return None

        except Exception as e:
            print(f"Either no such file {my_file_path_or_api} or Something error occurred in code : {e}")
            return None

## Document Loader With (.pdf, .txt, .json, .html, .csv and API) Supporting Extension
#### below you can create more concreate class if you wana to add more other Extention like docx and all

In [8]:
# Abstract base class for document loader
class AbstractDocumentLoader(ABC):
    @abstractmethod
    def load_document(self, my_file_path_or_api: str):
        pass

In [9]:
# Concrete class for loading CSV data
class CSVLoader(AbstractDocumentLoader):
    def load_document(self, my_file_path_or_api: str):
        from langchain_community.document_loaders.csv_loader import CSVLoader
        loader = CSVLoader(file_path=my_file_path_or_api)
        return loader.load()

In [10]:
# Concrete class for loading HTML data
class HTMLLoader(AbstractDocumentLoader):
    def load_document(self, my_file_path_or_api: str):
        from langchain_community.document_loaders import BSHTMLLoader
        loader = BSHTMLLoader(my_file_path_or_api)
        return loader.load()

In [11]:
# Concrete class for loading JSON data
class JSONLoader(AbstractDocumentLoader):
    def load_document(self, my_file_path_or_api: str):
        from langchain_community.document_loaders import JSONLoader
        loader = JSONLoader(
            file_path=my_file_path_or_api,
            jq_schema=".",
            text_content=False,
            json_lines=False
        )
        return loader.load()

In [12]:
# Concrete class for loading PDF data
class PDFLoader(AbstractDocumentLoader):
    def load_document(self, my_file_path_or_api: str):
        from langchain_community.document_loaders import PyPDFLoader
        loader = PyPDFLoader(my_file_path_or_api)
        return loader.load_and_split()

In [13]:
# Concrete class for loading TXT data
class TXTLoader(AbstractDocumentLoader):
    def load_document(self, my_file_path_or_api: str):
        from langchain.document_loaders import TextLoader
        loader = TextLoader(my_file_path_or_api)
        return loader.load()

In [14]:
# Concrete class for loading API data
class APILoader(AbstractDocumentLoader):
    def load_document(self, my_api_path: str):
        import requests
        from langchain.schema import Document
        response = requests.get(my_api_path)
        api_data = response.json()
        return [Document(page_content=str(api_data))]

### Below is Main class or we can say a factory class in by using below class we can call above classes with respect to document type, only this loader class will call.

#### Note - if you add above new concreate class don't furget to add .extension and its loader class in loader_mapping dict in below, don;t need to change anything else.

In [15]:
class DocumentHandler:
    
    def __init__(self):
        
        self.loader_mapping = {
            '.csv': CSVLoader(),
            '.html': HTMLLoader(),
            '.json': JSONLoader(),
            '.pdf': PDFLoader(),
            '.txt': TXTLoader(),
        }

    def load(self, my_file_path_or_api: str):
        try:
            if my_file_path_or_api.startswith("http"):
                loader = APILoader()
            else:
                file_extension = os.path.splitext(my_file_path_or_api)[1].lower()
                
                if file_extension in [".c", ".cpp", ".py", ".java", ".js", ".ts", ".cs", ".rb", ".php", ".html", ".css", ".swift", ".go", ".rs", ".kt", ".m", ".h", ".sh", ".pl", ".r", ".lua", ".asm", ".scala", ".sql", ".xml", ".json", ".yml", ".bat", ".ps1", ".vb", ".dart", ".erl", ".ex", ".f", ".ml", ".hs", ".jl", ".md", ".rkt", ".clj", ".v", ".vhd", ".pas", ".tsx", ".jsx"]:
                    file_extension = ".txt"
                    
                loader = self.loader_mapping[file_extension]
                if loader is None:
                    raise ValueError(f"Unsupported document type: {file_extension}")

            return loader.load_document(my_file_path_or_api)

        except ValueError as ve:
            print(f"ValueError : {ve}")
            return None

        except Exception as e:
            print(f"Either no such file {my_file_path_or_api} or Something error occurred in code : {e}")
            return None

### use case of above class

In [16]:
# Usage example

# let below two method i have choice that i can either clean the data or pass as it is without cleaning- 
document_cleaner = DocumentCleaner()

# Uncomment the line which document you wana to clean:
# document_cleaner.clean("csv data.csv")  # Cleaning CSV file
# document_cleaner.clean("html data.html")  # Cleaning HTML file
# document_cleaner.clean("json data.json")  # Cleaning JSON file
# document_cleaner.clean("txt data.txt")  # Cleaning TXT file

document_handler = DocumentHandler()

# Uncomment the line which document you wana to import:
# data = document_handler.load("csv data.csv")  # Loading CSV file
# data = document_handler.load("html data.html")  # Loading HTML file
# data = document_handler.load("json data.json")  # Loading JSON file
# data = document_handler.load("pdf data.pdf")  # Loading PDF file
# data = document_handler.load("txt data.txt")  # Loading TXT file
# data = document_handler.load("https://dummy-json.mock.beeceptor.com/todos")  # Loading API

In [17]:
class DocumentLoader_and_Cleaner:
    
    def __init__(self):
        
        self.cleaner_mapping = {
            '.csv': CSVCleaner(),
            '.html': HTMLCleaner(),
            '.json': JSONCleaner(),
            '.txt': TXTCleaner(),
        }
        
        self.loader_mapping = {
            '.csv': CSVLoader(),
            '.html': HTMLLoader(),
            '.json': JSONLoader(),
            '.pdf': PDFLoader(),
            '.txt': TXTLoader(),
        }

    def clean_and_load(self, my_file_path_or_api: list):
        try:
            if my_file_path_or_api.startswith("http"):
                loader = APILoader()
            
            else:    
                file_extension = os.path.splitext(my_file_path_or_api)[1].lower()
                
                if file_extension == ".pdf":
                    loader = self.loader_mapping[file_extension]
                    return loader.load_document(my_file_path_or_api)
                
                elif file_extension in [".c", ".cpp", ".py", ".java", ".js", ".ts", ".cs", ".rb", ".php", ".css", ".swift", ".go", ".rs", ".kt", ".m", ".h", ".sh", ".pl", ".r", ".lua", ".asm", ".scala", ".sql", ".xml", ".yml", ".bat", ".ps1", ".vb", ".dart", ".erl", ".ex", ".f", ".ml", ".hs", ".jl", ".md", ".rkt", ".clj", ".v", ".vhd", ".pas", ".tsx", ".jsx"]:
                    
                    file_extension = ".txt"
                
                loader = self.cleaner_mapping[file_extension]
                print(loader.clean_document(my_file_path_or_api))
                loader = self.loader_mapping[file_extension]
            
            if loader is None:
                raise ValueError(f"Unsupported File type: {file_extension}")
            
            return loader.load_document(my_file_path_or_api)
            
        except ValueError as ve:
            print(f"ValueError : {ve}")
            return None

        except Exception as e:
            print(f"Either no such file {my_file_path_or_api} or Something error occurred in code : {e}")
            return None

## Document Splitter With (.txt, .json, .html, .csv .pdf and Programming Langucage Extension's) Supporting Extension

In [18]:
import re
import copy
from langchain_text_splitters import RecursiveCharacterTextSplitter, HTMLHeaderTextSplitter, HTMLSectionSplitter,CharacterTextSplitter,Language
class Document:
    def __init__(self, page_content, metadata):
        self.page_content = page_content
        self.metadata = metadata

class DocumentSplitter:
    
    def __init__(self, chunk_size, chunk_overlap):
        
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        
        self.rcts_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=["\n\n", "\n", ".", "!", "?", ";", ",", ":", " ", "-", "_", "\uff0c", "\uff0e", "\u3001", "\u3002", "\u200b", "\u2028", "\u2029", ""],
            is_separator_regex=False
        )
    
    def __clean_documents(self, documents):
        cleaned_documents = []
        for document in documents:
            # Ensure page_content is a string and contains unwanted characters
            if isinstance(document.page_content, str):
                cleaned_text = re.sub(r'\s+', ' ', document.page_content)  # Replace multiple whitespace characters with a single space
                cleaned_text = cleaned_text.strip()  # Remove leading and trailing whitespace
                
                # Use deep copy to create a new Document object with cleaned content
                cleaned_document = copy.deepcopy(document)
                cleaned_document.page_content = cleaned_text
                cleaned_documents.append(cleaned_document)
        return cleaned_documents
    
    def split_by_rcts(self, documents):
        cleaned_documents = self.__clean_documents(documents)
        split_documents = []
        for document in cleaned_documents:
            # Ensure page_content is a string
            if isinstance(document.page_content, str):
                split_texts = self.rcts_splitter.create_documents([document.page_content])
                for split_doc in split_texts:
                    split_doc.metadata.update(document.metadata)
                    split_documents.append(split_doc)
            else:
                print(f"Skipping document with invalid content type: {type(document.page_content)}")
        return split_documents
    
    def split_by_char(self, documents):
        text_splitter = CharacterTextSplitter(
            separator="\n\n",chunk_size=self.chunk_size,chunk_overlap=self.chunk_overlap,length_function=len,is_separator_regex=False,)
        
        cleaned_documents = self.__clean_documents(documents)
        split_documents = []
        for document in cleaned_documents:
            # Ensure page_content is a string
            if isinstance(document.page_content, str):
                split_texts = text_splitter.create_documents([document.page_content])
                for split_doc in split_texts:
                    split_doc.metadata.update(document.metadata)
                    split_documents.append(split_doc)
            else:
                print(f"Skipping document with invalid content type: {type(document.page_content)}")
        return split_documents

    def split_by_code(self, documents):
        
        extension = documents[0].metadata["source"].split(".")[-1]
        language_mapping = {
            "cpp": Language.CPP, "go": Language.GO, "java": Language.JAVA,
            "kt": Language.KOTLIN, "js": Language.JS, "ts": Language.TS,
            "php": Language.PHP, "proto": Language.PROTO, "py": Language.PYTHON,
            "rst": Language.RST, "rb": Language.RUBY, "rs": Language.RUST,
            "scala": Language.SCALA, "swift": Language.SWIFT, "md": Language.MARKDOWN,
            "tex": Language.LATEX, "html": Language.HTML, "sol": Language.SOL,
            "cs": Language.CSHARP, "c": Language.C, "lua": Language.LUA,
            "pl": Language.PERL, "hs": Language.HASKELL
        }

        if extension in language_mapping:
            code_splitter = RecursiveCharacterTextSplitter.from_language(
                language=language_mapping[extension],
                chunk_size=self.chunk_size,
                chunk_overlap=self.chunk_overlap
            )
            
            split_documents = []
            for document in documents:
                if isinstance(document.page_content, str):
                    split_texts = code_splitter.create_documents([document.page_content])
                    for split_doc in split_texts:
                        split_doc.metadata.update(document.metadata)
                        split_documents.append(split_doc)
                else:
                    print(f"Skipping document with invalid content type: {type(document.page_content)}")
            return split_documents
        
        else:
            raise ValueError("Unsupported file format for code splitting.")

    def __split_by_html(self, documents, url, my_headers_to_split_on, splitter_class):
        """Helper function for HTML-based splitting."""
        html_splitter = splitter_class(headers_to_split_on=my_headers_to_split_on)

        if url:
            html_splits = html_splitter.split_text_from_url(url)
        else:
            html_splits = []
            for document in documents:
                split_texts = html_splitter.split_text(document.page_content)
                for split_doc in split_texts:
                    split_doc.metadata.update(document.metadata)
                    html_splits.append(split_doc)

        return self.split_by_rcts(html_splits)

    def split_by_html_header(self, documents=None, url=None, my_headers_to_split_on=None):
        return self.__split_by_html(documents, url, my_headers_to_split_on, HTMLHeaderTextSplitter)
    
    def split_by_html_section(self, documents=None, url=None, my_headers_to_split_on=None):
        return self.__split_by_html(documents, url, my_headers_to_split_on, HTMLSectionSplitter)

In [19]:
document_splitter = DocumentSplitter(50,10)

In [22]:
# but hear i don't have choice in one step i can clean and load the data
dlc = DocumentLoader_and_Cleaner()
doc_list = dlc.clean_and_load("short pdf.pdf")
# print(len(doc_list))
doc_list

[Document(metadata={'source': 'short pdf.pdf', 'page': 0}, page_content="The Impact of Artificial Intelligence on Modern Society  \nArtificial Intelligence (AI) is transforming various aspects of our daily lives. From smart assistants like \nSiri and Alexa to advanced algorithms that drive self -driving cars, AI has become an integral part of \nthe modern world.  \nIn healthcare, AI is used to analyze medical data, predict patient outcomes, and even assist in \ncomplex surgeries. In finance, algorithms are employed for fraud detection, investment analysis, and \nautomated trading. Moreover, AI is revolutionizing industri es such as retail, manufacturing, and \nlogistics by optimizing supply chains and improving customer experiences.  \nDespite its advantages, AI also poses challenges, including ethical concerns about privacy, job \ndisplacement, and decision -making transparency. As AI continues to evolve, it is crucial to address \nthese issues to ensure that the technology benefits s

In [23]:
split_doc = document_splitter.split_by_rcts(doc_list)
print(len(split_doc))
split_doc

42


[Document(metadata={'source': 'short pdf.pdf', 'page': 0}, page_content='The Impact of Artificial Intelligence on Modern'),
 Document(metadata={'source': 'short pdf.pdf', 'page': 0}, page_content='on Modern Society Artificial Intelligence (AI) is'),
 Document(metadata={'source': 'short pdf.pdf', 'page': 0}, page_content='(AI) is transforming various aspects of our daily'),
 Document(metadata={'source': 'short pdf.pdf', 'page': 0}, page_content='our daily lives'),
 Document(metadata={'source': 'short pdf.pdf', 'page': 0}, page_content='. From smart assistants like Siri and Alexa to'),
 Document(metadata={'source': 'short pdf.pdf', 'page': 0}, page_content='Alexa to advanced algorithms that drive self'),
 Document(metadata={'source': 'short pdf.pdf', 'page': 0}, page_content='self -driving cars'),
 Document(metadata={'source': 'short pdf.pdf', 'page': 0}, page_content=', AI has become an integral part of the modern'),
 Document(metadata={'source': 'short pdf.pdf', 'page': 0}, page_conten

In [24]:
import os
import psycopg2
import json
import uuid
from langchain.vectorstores import PGVector
from langchain.embeddings import HuggingFaceInstructEmbeddings


class DocumentEmbedding:
    
    def __init__(self, documents):
        self.instructor_embeddings = HuggingFaceInstructEmbeddings(
            model_name="hkunlp/instructor-xl",
            model_kwargs={"device": "cpu"}
        )
        self.documents = documents

    def get_embeddings(self):
        embeddings = []
        for document_chunk in self.documents:
            embedding = self.instructor_embeddings.embed_documents([document_chunk.page_content])[0]
            embedding_data = {
                "id": str(uuid.uuid4()),
                "content": document_chunk.page_content,
                "meta": document_chunk.metadata,
                "embedding": embedding
            }
            embeddings.append(embedding_data)
        return embeddings

In [25]:
document_embed = DocumentEmbedding(split_doc)

c:\Users\SHANOOR\AppData\Local\Programs\Python\Python312\Lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


c:\Users\SHANOOR\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\SHANOOR\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\models\Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions t

In [26]:
get_embed = document_embed.get_embeddings()

In [27]:
print(type(split_doc))
print(type(split_doc[0]))

<class 'list'>
<class 'langchain_core.documents.base.Document'>


In [28]:
class FileEmbeddingStorage:
    
    def __init__(self, embeddings, file_path):
        self.embeddings = embeddings
        self.file_path = file_path
    
    def store(self):
        embeddings_file = os.path.join(self.file_path, 'embeddings.txt')
        with open(embeddings_file, 'w') as file:
            for embedding_data in self.embeddings:
                file.write(json.dumps(embedding_data) + '\n')
        print(f"Embeddings saved successfully in {embeddings_file}")

In [29]:
file_embed_store = FileEmbeddingStorage(get_embed,"embeddings_output")

In [35]:
class DatabaseEmbeddingStorage:
    
    def __init__(self, embeddings, my_split_doc,instance_class,mydbname="Sample_DataBase", myuser="postgres", mypassword="root", myhost="localhost", myport="5432"):
        self.embeddings = embeddings
        self.instance = instance_class
        self.split_doc = my_split_doc
        self.dbname = mydbname
        self.user = myuser
        self.password = mypassword
        self.host = myhost
        self.port = myport
        self.conn = self._make_db_connection()
    
    def _make_db_connection(self):
        try:
            conn = psycopg2.connect(
                dbname=self.dbname,  
                user=self.user,           
                password=self.password,  
                host=self.host,          
                port=self.port
            )
            return conn
        except psycopg2.OperationalError as e:
            print(f"Error: Could not connect to the database. Reason: {e}")
            return None

    def _create_table(self, table_query):
        cursor = self.conn.cursor()
        cursor.execute(table_query)
        self.conn.commit()
        cursor.close()

    def store(self):
        if not self.conn:
            print("Database connection failed.")
            return

        create_json_table_query = """
        CREATE TABLE IF NOT EXISTS embeddings_json (
            id TEXT PRIMARY KEY,
            meta TEXT,
            content TEXT NOT NULL,
            embedding JSONB NOT NULL
        );
        """
        self._create_table(create_json_table_query)

        insert_query = """
        INSERT INTO embeddings_json (id, meta, content, embedding)
        VALUES (%s, %s, %s, %s)
        ON CONFLICT (id) DO NOTHING;
        """
        cursor = self.conn.cursor()
        for embedding_data in self.embeddings:
            cursor.execute(insert_query, (
                embedding_data["id"], 
                json.dumps(embedding_data["meta"]), 
                embedding_data["content"], 
                json.dumps(embedding_data["embedding"])
            ))
            self.conn.commit()
        
        cursor.close()
        self.conn.close()
        print("Embeddings inserted into PostgreSQL successfully!")

    def store_in_pgvector(self, my_collection_name):
        self.conn = self._make_db_connection()
        if not self.conn:
            print("Database connection failed.")
            return
        
        create_pgvector_table_query = """
        CREATE TABLE IF NOT EXISTS embeddings_pgvector (
            id TEXT PRIMARY KEY,
            meta TEXT,
            content TEXT NOT NULL,
            embedding vector(768)
        );
        """
        self._create_table(create_pgvector_table_query)

        CONNECTION_STRING = PGVector.connection_string_from_db_params(
            driver="psycopg2",
            host=self.host,
            port=self.port,
            database=self.dbname,
            user=self.user,
            password=self.password
        )
        
        db = PGVector.from_documents(
            embedding=self.instance.instructor_embeddings,
            documents= self.split_doc,
            collection_name=my_collection_name,
            connection_string=CONNECTION_STRING,
            pre_delete_collection=True
        )

        self.conn.close()
        print(f"Embeddings inserted into PGVector collection '{my_collection_name}' successfully!")
        return db

In [36]:
db_obj = DatabaseEmbeddingStorage(get_embed,split_doc,document_embed,"Sample_DataBase","postgres","root","localhost","5432")

In [37]:
print(type(get_embed[0]))
print(get_embed[0])


<class 'dict'>
{'id': '22b50637-eddd-439c-9d9d-97bbeab36da1', 'content': 'The Impact of Artificial Intelligence on Modern', 'meta': {'source': 'short pdf.pdf', 'page': 0}, 'embedding': [-0.019584836438298225, 0.00015889934729784727, 0.039740949869155884, -0.05959833040833473, -0.10919196903705597, -0.04749070480465889, -0.0501183345913887, -0.04814533516764641, -0.05852341279387474, -0.06992729753255844, 0.03585956618189812, 0.013958292081952095, -0.034639738500118256, -0.09370286762714386, 0.01548604853451252, -0.03756038472056389, -0.005083034746348858, -0.07230103015899658, -0.01091417670249939, -0.01570998504757881, -0.027006128802895546, 0.02054597996175289, -0.02925829030573368, 0.021313119679689407, 0.004738526418805122, -0.07115960866212845, -0.028102770447731018, 0.025967972353100777, -0.008715696632862091, 0.023442978039383888, 0.011703362688422203, -0.0200587697327137, 0.028940964490175247, -0.019682524725794792, 0.033738572150468826, -0.05096295103430748, 0.0277838818728923

In [38]:
db_obj.store()

Embeddings inserted into PostgreSQL successfully!


In [39]:
db_obj.store_in_pgvector("Essay on If i become prime minister")

c:\Users\SHANOOR\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\vectorstores\pgvector.py:487: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata.Please note that filtering operators have been changed when using JSOB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create adb migration for your metadata column to be JSONB and update your queries to use the new operators. 
  store = cls(
Collection not found


Embeddings inserted into PGVector collection 'Essay on If i become prime minister' successfully!
